In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor

/home/yuru/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def dcg(r):
    r = np.asfarray(r)[:10]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
    return 0.

def ndcg(r):
    dcg_max = dcg(sorted(r, reverse=True))
    if not dcg_max:
        return 0.
    return dcg(r) / dcg_max

In [5]:
train_pool = pd.read_csv('data/train.csv', encoding='utf-8')
test_pool = pd.read_csv('data/test.csv', encoding='utf-8')

Напишем функции, генерирующие простые признаки основанные на пересечении триграмм между запросом и названием организации

In [21]:
def get_trigrams(string):
    #string = '^^' + string + '$$'
    trigrams = set()
    trigrams_count = 0
    
    for i in range(len(string) - 2):
        trigrams.add(string[i:i+3])
        trigrams_count += 1
        
    return trigrams, trigrams_count

def common_trigrams_factors(query, org_name):
    query_trigrams, query_trigrams_count = get_trigrams(query)
    org_name_trigrams, org_name_trigrams_count = get_trigrams(org_name)

    factors = [float(len(query_trigrams.intersection(org_name_trigrams)))]

    factors.append(0. if query_trigrams_count == 0. else 0.1 + factors[0] / query_trigrams_count)
    factors.append(0. if org_name_trigrams_count == 0. else 0.1 + factors[0] / org_name_trigrams_count)    
    
    return factors

Посчитаем данные факторы для каждого файла

In [22]:
def calc_trigram_factors(row):
    return pd.Series(common_trigrams_factors(row.query, row.org_name))
    
train_factors = train_pool.apply(calc_trigram_factors, axis=1)
test_factors = test_pool.apply(calc_trigram_factors, axis=1)

Обучаем модель

In [23]:
clf = XGBRegressor(n_estimators=100)
clf.fit(train_factors.values, train_pool[['relevance']].values)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

Добавляем колонку с предсказанной релевантностью

In [24]:
test_pool['relevance'] = clf.predict(test_factors.values)

Сортируем организации по предсказанной релевантности для каждой организации и записываем их в файл

In [25]:
test_pool.sort_values(['query_id', 'relevance'], 
                      ascending=[True, False])[['query_id', 'org_id']].to_csv('result.csv', index=None)

In [13]:
test_pool

,query_id,query,region,org_name,org_id,window_center,window_size,relevance
0,13,цирк,143,Національний цирк України,1037098182,"30.450273,50.480808","0.329557,0.333847",0.140718
1,13,цирк,143,"Цирк Kobzov, головний офіс",1290917418,"30.450273,50.480808","0.329557,0.333847",0.124156
2,53,магазины продуктов,27555,Империя,1068980562,"34.414789,45.715563","0.012058,0.002735",0.069919
3,53,магазины продуктов,27555,Фора,1140113561,"34.414789,45.715563","0.012058,0.002735",0.069919
4,56,магазины продуктов,10347,Магазин Продукти,1135260095,"33.443379,47.992092","0.011033,0.001702",0.057006
5,56,магазины продуктов,10347,АТБ,1192031255,"33.443379,47.992092","0.011033,0.001702",0.069919
6,56,магазины продуктов,10347,Ехо,1248487339,"33.443379,47.992092","0.011033,0.001702",0.069919
7,56,магазины продуктов,10347,Магазин Економний,1357365016,"33.443379,47.992092","0.011033,0.001702",0.058938
8,56,магазины продуктов,10347,АТБ,1063407364,"33.443379,47.992092","0.011033,0.001702",0.069919
9,56,магазины продуктов,10347,АТБ,1063990185,"33.443379,47.992092","0.011033,0.001702",0.069919


In [20]:
s1 = 'fkg'
s2 = 'fkg'
common_trigrams_factors(s1, s2)

[5.0, 1.1, 1.1]

In [19]:
float(len(['sdf', 'fdd']))

2.0